<a href="https://colab.research.google.com/github/DongUk-Park/RnD/blob/main/POI/NAIS/DataPreprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import json
import csv
import os
import pandas as pd
import numpy as np
import random

#Data Untils

## RAW Data 생성

### Check_in Data 생성
리뷰 데이터로 체크인 데이터(user,business_id)를 만든다 <br>
원하는 데이터 정보 추가,삭제 : 13, 21 수정

In [67]:
# JSON 파일 경로
json_file_path = '/content/drive/MyDrive/dataset/yelp_dataset/review.json'

# CSV 파일 경로
csv_file_path = '/content/checkin.csv'

# CSV 파일을 쓰기 모드로 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    # CSV 라이터 생성
    csv_writer = csv.writer(csv_file)

    # CSV 파일 헤더 작성
    csv_writer.writerow(['user_id', 'business_id'])

    # JSON 파일을 한 줄씩 읽어서 처리
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            data = json.loads(line)  # JSON 데이터 파싱

            # user_id와 business_id 추출
            user_id = data['user_id']
            business_id = data['business_id']

            # CSV 파일에 데이터 작성
            csv_writer.writerow([user_id, business_id])

print("checkin_data 생성 완료.")


checkin_data 생성 완료.


In [68]:
"""
데이터 개수(row 수) 확인
"""

# CSV 파일 경로
csv_file_path = '/content/checkin.csv'

row_count = 0

# CSV 파일을 읽기 모드로 열고 행 수를 센다
with open(csv_file_path, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # 헤더 행을 건너뜁니다.
    for row in csv_reader:
        row_count += 1

print(f"CSV 파일의 행 수: {row_count}")

CSV 파일의 행 수: 6990280


### Business_info Data 생성
Business 데이터로 가게의 위치를 담은 데이터(business_id,location, city)를 만든다 <br>
원하는 데이터 정보 추가,삭제 : 13, 21 수정

In [69]:
# JSON 파일 경로
json_file_path = '/content/drive/MyDrive/dataset/yelp_dataset/business.json'

# CSV 파일 경로
csv_file_path = '/content/business_info.csv'

# CSV 파일을 쓰기 모드로 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    # CSV 라이터 생성
    csv_writer = csv.writer(csv_file)

    # CSV 파일 헤더 작성
    csv_writer.writerow(['business_id','latitude ','longitute', 'city'])

    # business_id 를 int형으로 변환
    idx = 0

    # JSON 파일을 한 줄씩 읽어서 처리
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            data = json.loads(line)  # JSON 데이터 파싱

            # user_id와 business_id 추출
            business_id = data['business_id']
            latitude = data['latitude']
            longitude = data['longitude']
            city = data['city']

            # CSV 파일에 데이터 작성
            csv_writer.writerow([business_id, latitude, longitude, city])
            idx += 1

print("CSV 파일 생성 완료.")


CSV 파일 생성 완료.


In [70]:
"""
데이터 개수(row 수) 확인
"""
# CSV 파일 경로
csv_file_path = '/content/business_info.csv'

row_count = 0

# CSV 파일을 읽기 모드로 열고 행 수를 센다
with open(csv_file_path, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # 헤더 행을 건너뜁니다.
    for row in csv_reader:
        row_count += 1

print(f"CSV 파일의 행 수: {row_count}")

CSV 파일의 행 수: 150346


## 사용자 방문데이터(위치포함) 생성
위에서 생성한 데이터 두개를 합친 방문데이터 생성 <br>
checkin + business_info => location과 city 확인<br>
Philadelphia 도시에 대한 데이터만 남김

In [71]:
# business_info.csv 파일 경로
business_info_file = '/content/business_info.csv'

# checkin_data.csv 파일 경로
checkin_data_file = '/content/checkin.csv'

# 새로운 CSV 파일 경로 (결과 저장용)
output_file = '/content/checkin_data_in_philadelphia.csv'

# business_id를 키로 한 dictionary 생성
business_location = {}
with open(business_info_file, 'r', newline='') as business_file:
    csv_reader = csv.reader(business_file)
    next(csv_reader)  # 헤더 행 건너뛰기
    for row in csv_reader:
        business_id, latitude, longitude, city = row[0], row[1], row[2], row[3].lower() #city : 소문자로 받음
        business_location[business_id] = (latitude, longitude, city)

count = 0 # 잘 다 들어갔는지 체크
mcount = 0 # 잘 안들어간거 있는지 체크
not_phila = 0

# 새로운 CSV 파일(output_file : checkin_data_in_philadelphia)을 작성하여 business_id, latitude, longitude를 추가
with open(checkin_data_file, 'r', newline='') as checkin_file, open(output_file, 'w', newline='') as output_csv:
    csv_reader = csv.reader(checkin_file)
    csv_writer = csv.writer(output_csv)

    # 헤더 행 작성
    header = next(csv_reader)
    header.extend(['business_id', 'latitude', 'longitude', 'city'])
    csv_writer.writerow(header)

    for row in csv_reader:
        business_id = row[1]
        if business_id in business_location:
            latitude, longitude, city = business_location[business_id]

            # city가 "philadelphia"인 경우에만 데이터를 CSV 파일에 작성
            if "philadelphia" in city: #philadelphia 에서 오타가 존재할 수 있기 때문에 "phila"만 검사
              row.extend([latitude, longitude, city])
              csv_writer.writerow(row)
              count += 1
            else:
              not_phila += 1
        else:
            # 해당 business_id가 business_info.csv에 없을 경우, 스킵
            mcount += 1
            continue

print("CSV 파일에 위치 정보가 추가되었습니다.")
print(count, mcount, not_phila) #philadelphia, 에러, 다른도시


CSV 파일에 위치 정보가 추가되었습니다.
968050 0 6022230


user_id 순으로 정렬

In [72]:
# CSV 파일 경로
input_file = '/content/checkin_data_in_philadelphia.csv'
output_file = '/content/checkin_data_final.csv'

# CSV 파일을 읽고 데이터를 리스트로 저장
data = []
with open(input_file, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)  # 헤더 행
    #data.append(header)
    for row in csv_reader:
        data.append(row)

# user_id를 기준으로 데이터를 정렬
data.sort(key=lambda x: x[0])  # 여기서 0은 user_id 열을 가리킵니다. 0부터 시작하면 첫 번째 열입니다.

# user_id를 정수형으로 변환
idx = 0
before_user_id = data[0][0]
for i in data:
  if i[0] == before_user_id:
    i[0] = idx
  else:
    idx += 1
    before_user_id = i[0]
    i[0] = idx


# business_id 기준으로 데이터를 정렬
data.sort(key=lambda x: x[1])  # 여기서 0은 user_id 열을 가리킵니다. 0부터 시작하면 첫 번째 열입니다.
print(data[:3])

# business_id 정수형으로 변환
idx = 0
before_business_id = data[1][0]
for i in data:
  if i[1] == before_business_id:
    i[1] = idx
  else:
    idx += 1
    before_business_id = i[1]
    i[1] = idx

# 다시 유저 번호 기준으로 정렬
data.sort(key=lambda x: x[0])
print(data[:3])

# 정렬된 데이터를 새로운 파일에 저장
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(data)

print("CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.")

[[55363, '--OS_I7dnABrXvRCCuWOGQ', '40.0281603', '-75.0338026', 'philadelphia'], [65734, '--OS_I7dnABrXvRCCuWOGQ', '40.0281603', '-75.0338026', 'philadelphia'], [123557, '--OS_I7dnABrXvRCCuWOGQ', '40.0281603', '-75.0338026', 'philadelphia']]
[[0, 1482, '40.0741872', '-75.2030839', 'philadelphia'], [0, 8855, '39.9544408', '-75.1704266', 'philadelphia'], [1, 7951, '39.9418368', '-75.1538396', 'philadelphia']]
CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.


In [79]:
data.sort(key=lambda x: x[1])

column =['user_id', 'business_id', 'latitude', 'longitude', 'city']
df = pd.DataFrame(data, columns=column)
# 중복되지 않는 '키' 값을 찾고 필터링
result = df.drop_duplicates('business_id', keep='first')[['business_id', 'latitude', 'longitude', 'city']].values.tolist()
print(len(result))

with open('/content/business_info_in_philadelphia.csv', 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(result)

14586


In [76]:
"""
train에 사용될 history 데이터 (2차원 리스트)
인덱스 번호 : user_Id
인덱스 value(list형식) : 해당 uesr가 방문한 business_id 리스트
"""
data.sort(key=lambda x: x[0])
history_list = []
tmp = []
before_user_id = 0
for idx, i in enumerate(data):
  if i[0] == before_user_id:
    tmp.append(i[1])
  else:
    if len(tmp) >= 10: # 방문 횟수가 10회가 넘는 유저만 append
      history_list.append(tmp)

    tmp = []
    tmp.append(i[1])
    before_user_id += 1

  if idx == len(data) - 1:
    history_list.append(tmp)

output_file = '/content/history_list.csv'
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(history_list)

## Test,Validation,Train Data 생성

In [77]:
"""
위 history_list를 train, validation, test 데이터셋으로 분리시킴
"""

train_ratio = 0.6
test_ratio = 0.2
validation_ratio = 0.2

train_dataset = []
test_dataset = []
validation_dataset = []

train_negative_dataset = [] # business_id 개수 : 150346개
validation_negative_dataset = []
test_negative_dataset = []


#아래의 결과를 csv 파일로 저장할 경로, 총 6개의 파일 생성
output_train = '/content/history_train.csv'
output_validation = '/content/history_validation.csv'
output_test = '/content/history_test.csv'


for user, history in enumerate(history_list):
  ### 한 유저에 대해 train,validation,test dataset 생성
  total_samples = len(history) # 방문 횟수 저장
  random.shuffle(history) # 방문 기록 순서를 랜덤하게 섞어준다.

  train_size = int(total_samples * train_ratio)
  validation_size = int(total_samples * validation_ratio)

  list_test = history[(train_size + validation_size):]
  list_validation = history[train_size: (train_size + validation_size)]
  list_train = history[:train_size]


  train_dataset.append(list_train)
  validation_dataset.append(list_validation)
  test_dataset.append(list_test)

with open(output_train, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(train_dataset)
with open(output_validation, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(validation_dataset)
with open(output_test, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(test_dataset)


In [78]:
print(len(train_dataset), train_dataset[:5])
print(len(validation_dataset), validation_dataset[:5])
print(len(test_dataset), test_dataset[:5])


# print(len(train_negative_dataset), train_negative_dataset[:5])
# print(len(validation_negative_dataset), validation_negative_dataset[:5])
# print(len(test_negative_dataset), test_negative_dataset[:5])

15919 [[11095, 11026, 1718, 5402, 13512, 1123, 6681, 4386, 11749, 7996], [13674, 14107, 10244, 7590, 13072, 12784, 8145, 3639, 8623, 13796], [1194, 13317, 10751, 11026, 12027, 2916, 442, 10011], [4589, 10933, 6605, 3951, 11013, 2420, 8209], [5152, 71, 3142, 8, 6932, 3297]]
15919 [[8083, 1836, 6171], [6663, 2296, 5780], [7001, 12202], [1680, 6287], [11850, 5034]]
15919 [[4479, 20, 13070, 821], [9623, 12806, 2296, 8805, 14445], [442, 7879, 8913, 2428], [4589, 9645, 9213], [13564, 10824, 14525]]


## 지역 임베딩 데이터 생성
NAIS 코드 완성 이후 진행 예정

## 거리 데이터 생성

In [61]:
# Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # 지구 반지름 (km)
    return c * r

In [62]:
# 데이터 불러오기
header = ["user_id", "item", "latitude", "longitude", "city"]
checkin_data_with_distance = pd.read_csv('checkin_data_final.csv', names=header)

checkin_data_with_distance['closest_distance'] = 0

before_userID = None
result = []

#거리 계산
for index, user_row in checkin_data_with_distance.iterrows():
    userID = user_row['user_id']
    if before_userID != userID:
      same_user_df = checkin_data_with_distance[checkin_data_with_distance['user_id'] == userID]
      before_userID = userID
    if same_user_df.shape[0] < 10:
      continue # 10 보다 작은건 건너띔 -> 실제로 쓰는 데이터는

    user_item = user_row['item']
    user_lat = user_row['latitude']
    user_lon = user_row['longitude']
    closest_distance = 9999999  # 초기값을 무한대로 설정

    for poi_index, poi_row in same_user_df.iterrows():
        poi_item = poi_row['item']
        poi_lat = poi_row['latitude']
        poi_lon = poi_row['longitude']

        # Haversine 거리 계산
        distance = haversine(user_lat, user_lon, poi_lat, poi_lon)

        # 현재 POI와의 거리가 가장 작은지 확인
        if distance < closest_distance and distance != 0:
            closest_distance = distance
            #print(closest_distance)

    # 방문한 Poi가 하나밖에 없다면 거리 : 0, 근데 그런거 없음
    if closest_distance == 9999999:
      closest_distance = 0


    #print(f"{index} : {closest_distance}")
    result.append(closest_distance)

print(result)

KeyboardInterrupt: ignored

In [ ]:
# 리스트를 Pandas DataFrame으로 변환
data = pd.DataFrame(result, columns=['distance'])

# 결과 출력
print(data)
data.to_csv('/content/distance_list.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data['rounded_distance'] = data['distance'].round(1)
# 로그를 취할 열 추가(ground_truth)
data['log_rounded_distance'] = np.log2(data['rounded_distance'])
data = data[data['log_rounded_distance'] != -np.inf] #반올림 결과가 0인 데이터 95183개 삭제

distance_counts = data['log_rounded_distance'].value_counts() # 거리별 방문횟수 저장
distance_counts.iloc[0] += 95183 # 0인 애들을 카운트가 가장 많은, 즉 가장 가까운 곳으로 다 더해줌

total_counts = distance_counts.sum()

print(distance_counts)
print("Counts의 합:", total_counts)

x1 = distance_counts.index # log2(distance)
y1 = np.log2(distance_counts.values / total_counts)

print(x1)
print(y1)

# 그래프 그리기
plt.figure(figsize=(10, 6))  # 그래프 크기 설정 (선택 사항)
plt.scatter(x1, y1)
plt.xlabel('Rounded Closest Distance(log sacale)')
plt.ylabel('Prob(log sacale)')
plt.title('Distribution of Rounded Closest Distance(log scale)')
#plt.ylim(0, 0.005)
#plt.xlim(0,1)
plt.xticks(rotation=45)  # x축 레이블 회전 (선택 사항)
plt.show()

In [ ]:
import torch
import torch.nn

X = torch.tensor(x1, dtype=torch.float32).view(-1, 1)
y = torch.tensor(y1, dtype=torch.float32).view(-1, 1)  # 확률, 2D 텐서로 변환

In [ ]:
# 모델 정의
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.w1 = nn.Parameter(torch.randn(1, requires_grad=True))
        self.w2 = nn.Parameter(torch.randn(1, requires_grad=True))

    def forward(self, x):
        return self.w1 + self.w2 * x

model = MyModel()
criterion = nn.MSELoss()  # 평균 제곱 오차 손실 함수
optimizer = optim.SGD(model.parameters(), lr=0.01)  # 확률적 경사 하강법 사용

# 학습
num_epochs = 1000  # 학습 에포크 수
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    #if (epoch + 1) % 100 == 0:
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# 학습된 파라미터 출력
print(f'학습된 파라미터 w1: {model.w1.item():.4f}, w2: {model.w2.item():.4f}')

In [ ]:
# 선형 회귀 결과를 그래프로 표현
plt.figure(figsize=(10, 6))

plt.scatter(x1, y1)

# 선형 회귀 결과
plt.plot(X, model.w1.item() + model.w2.item() * X , color='red', label='Linear Regression')

plt.xlabel('Rounded Closest Distance')
plt.ylabel('Prob')
plt.title('y에 log 씌운 것')
plt.xticks(rotation=45)

plt.legend()
plt.show()

In [ ]:
b = model.w2.item()
#a = np.exp(model.w1.item())
a = 2 ** model.w1.item()

print(a,b)

x1 = np.arange(40)
x = x1 + 0.5
#print(x)
y = a * (x**b)
print(y)